In [1]:
%matplotlib inline

import os

import keras as keras
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.datasets import cifar10
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing.image import ImageDataGenerator

random_state = 42

C:\Users\yanor\Anaconda3\envs\matsuo\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Section4 実装②

### 4.1 CIFAR10のデータをCNNでクラス分類②

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255
y_train = np.eye(10)[y_train.astype('int32').flatten()]

x_test = x_test.astype('float32') / 255
y_test = np.eye(10)[y_test.astype('int32').flatten()]

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=10000)

Section3の学習内容も踏まえて、CIFAR10のクラス分類を行う。

まず、モデルの作成を行う。

In [3]:
model = Sequential()

model.add(Conv2D(6, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal', input_shape=(32, 32, 3)))  # 32x32x3 -> 28x28x6
model.add(MaxPooling2D(pool_size=(2, 2)))  # 28x28x6 -> 14x14x6
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal'))  # 14x14x6 -> 10x10x16
model.add(MaxPooling2D(pool_size=(2, 2)))  # 10x10x16 -> 5x5x16

model.add(Flatten())  # 5x5x16 -> 400
model.add(Dense(120, activation='relu',
                kernel_initializer='he_normal'))  # 400 ->120
model.add(Dense(84, activation='relu', kernel_initializer='he_normal'))  # 120 ->84
model.add(Dense(10, activation='softmax'))  # 84 ->10

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

次に、Section③で学習したDataAugumentationや画像データの正規化を学習に反映させてみる。

kerasのImageDataGeneratorを使用する。

In [4]:
datagen = ImageDataGenerator(
    width_shift_range=0.2,  # 3.1.1 左右にずらす
    height_shift_range=0.2,  # 3.1.2 上下にずらす
    horizontal_flip=True,  # 3.1.3 左右反転
    # 3.2.1 Global Contrast Normalization (GCN) (Falseに設定しているのでここでは使用していない)
    samplewise_center=False,
    samplewise_std_normalization=False,
    zca_whitening=False)  # 3.2.2 Zero-phase Component Analysis (ZCA) Whitening (Falseに設定しているのでここでは使用していない)

In [5]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),
                    steps_per_epoch=x_train.shape[0] // 100, epochs=30, validation_data=(x_valid, y_valid))

Epoch 1/30
400/400 [==============================] - 257s 642ms/step - loss: 1.9186 - acc: 0.2872 - val_loss: 1.6000 - val_acc: 0.4172
Epoch 2/30
400/400 [==============================] - 266s 665ms/step - loss: 1.6674 - acc: 0.3889 - val_loss: 1.4742 - val_acc: 0.4599
Epoch 3/30
400/400 [==============================] - 218s 545ms/step - loss: 1.5932 - acc: 0.4205 - val_loss: 1.4268 - val_acc: 0.4890
Epoch 4/30
400/400 [==============================] - 167s 419ms/step - loss: 1.5405 - acc: 0.4398 - val_loss: 1.4361 - val_acc: 0.4784
Epoch 5/30
400/400 [==============================] - 191s 479ms/step - loss: 1.5006 - acc: 0.4564 - val_loss: 1.4170 - val_acc: 0.4965
Epoch 6/30
400/400 [==============================] - 202s 504ms/step - loss: 1.4732 - acc: 0.4679 - val_loss: 1.3284 - val_acc: 0.5248
Epoch 7/30
400/400 [==============================] - 229s 572ms/step - loss: 1.4376 - acc: 0.4787 - val_loss: 1.3203 - val_acc: 0.5242
Epoch 8/30
400/400 [============================